In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys

import torchaudio
import tgt
import os
import numpy as np
from tqdm import tqdm
import librosa
import glob
import soundfile

sys.path.append("../")
from pyfiles.feature_extractor import WavLMExtractor
from vocos.feature_extractors import MelSpectrogramFeatures

# sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
# from sound import play_audio

def trim_audio_and_save(path, fs=16000, savepath="temp.wav", trim_threshold_in_db=30, trim_frame_size=2048, trim_hop_size=512):
    audio, _ = librosa.load(path, sr=fs)
    audio, _ = librosa.effects.trim(
        audio,
        top_db=trim_threshold_in_db,
        frame_length=trim_frame_size,
        hop_length=trim_hop_size,
    )
    torchaudio.save(savepath, torch.tensor(audio).unsqueeze(0), fs)
    return 
def get_mel(path):
    y, sr = torchaudio.load(path)
    if fs!=sr:
        if y.size(0) > 1:  # mix to mono
            y = y.mean(dim=0, keepdim=True)
        y = torchaudio.functional.resample(y, orig_freq=sr, new_freq=fs)
    mel = melgen(y[0])
    return mel

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
data_dir = "/mntcephfs/lab_data/shoinoue/"
dataset_dir = data_dir + "Dataset/PD-AST/SLT/"
fs = 16000
trim_threshold_in_db = 30

melgen = MelSpectrogramFeatures(sample_rate=fs)
wavlm = WavLMExtractor()
speakers = ["Hindi"]
# speakers = [os.path.basename(a) for a in glob.glob(dataset_dir + "*")]
# speakers.sort()

In [7]:
save = True
tempfile = "temp2.wav"

for spk in speakers:
    print(spk)
    wavlm_dir = f"{dataset_dir}{spk}/wavlm/"
    mel_dir = f"{dataset_dir}{spk}/mel/"
    os.makedirs(wavlm_dir, exist_ok=True)
    os.makedirs(mel_dir, exist_ok=True)
    filenames = glob.glob(dataset_dir + spk + "/wav/*")
    filenames.sort()
    for path in tqdm(filenames):
        trim_audio_and_save(path, savepath=tempfile, trim_threshold_in_db=trim_threshold_in_db)
        
        # wavlm
        embedding = wavlm.get_feature(tempfile).T.detach().cpu().numpy()
        savepath = wavlm_dir + os.path.basename(path)[:-4] + ".npy"
        if save:
            np.save(savepath, embedding)
            
        # mel
        mel = get_mel(tempfile)
        savepath = mel_dir + os.path.basename(path)[:-4] + ".npy"
        if save:
            np.save(savepath, mel)

Hindi


100%|██████████| 1132/1132 [03:08<00:00,  6.01it/s]
